# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-0.33,100,100,4.63,CA,1730068597
1,1,ushuaia,-54.8000,-68.3000,7.81,57,75,1.03,AR,1730068598
2,2,praia da vitoria,38.7333,-27.0667,18.14,77,75,3.60,PT,1730068599
3,3,kalmunai,7.4167,81.8167,26.24,84,49,0.48,LK,1730068600
4,4,blackmans bay,-43.0167,147.3167,12.07,62,13,1.79,AU,1730068601


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:

# Create the map plot using points
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    frame_width=800,
    frame_height=600,
   
)
  
 
# Display the plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df[(city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,callao,-12.0667,-77.1500,19.16,72,0,6.69,PE,1730068623
40,40,tura,25.5198,90.2201,20.99,86,0,1.66,IN,1730068643
42,42,loreto,22.2667,-101.9667,21.03,30,0,6.55,MX,1730068646
48,48,san patricio,28.0170,-97.5169,28.97,43,0,5.30,US,1730068654
69,69,port elizabeth,-33.9180,25.5701,17.95,58,0,4.12,ZA,1730068680
...,...,...,...,...,...,...,...,...,...,...
498,498,namest nad oslavou,49.2073,16.1585,11.35,93,0,2.68,CZ,1730069190
504,504,los alamos,35.8881,-106.3070,22.23,31,0,2.06,US,1730069197
517,517,al burayqah,30.4062,19.5739,21.03,53,0,3.92,LY,1730069211
525,525,gaiman,-43.2897,-65.4929,19.11,27,0,6.03,AR,1730069220


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    'City': filtered_city_data_df['City'],
    'Country': filtered_city_data_df['Country'],
    'Lat': filtered_city_data_df['Lat'],
    'Lng': filtered_city_data_df['Lng'],
    'Humidity': filtered_city_data_df['Humidity']
})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
22,callao,PE,-12.0667,-77.1500,72,
40,tura,IN,25.5198,90.2201,86,
42,loreto,MX,22.2667,-101.9667,30,
48,san patricio,US,28.0170,-97.5169,43,
69,port elizabeth,ZA,-33.9180,25.5701,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
  
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[10, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
callao - nearest hotel: 
tura - nearest hotel: 
loreto - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
port elizabeth - nearest hotel: 
aguililla - nearest hotel: No hotel found
port isabel - nearest hotel: 
siwa oasis - nearest hotel: 
la passe - nearest hotel: 
ha'il - nearest hotel: 
kish - nearest hotel: 
yarmouth - nearest hotel: 
salalah - nearest hotel: 
coahuayana de hidalgo - nearest hotel: No hotel found
hamilton - nearest hotel: 
sinop - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
mazatlan - nearest hotel: 
deqen - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: 
hami - nearest hotel: 
brookings - nearest hotel: 
balkanabat - nearest hotel: 
talcahuano - nearest hotel: 
geraldton - nearest hotel: 
riohacha - nearest hotel: 
al jawf - nearest hotel: No hotel found
punta del este - nearest hotel: 
cabo san lucas - nearest hotel: 
acapulco de juarez -

,City,Country,Lat,Lng,Humidity,Hotel Name
22,callao,PE,-12.0667,-77.1500,72.0,
40,tura,IN,25.5198,90.2201,86.0,
42,loreto,MX,22.2667,-101.9667,30.0,No hotel found
48,san patricio,US,28.0170,-97.5169,43.0,No hotel found
69,port elizabeth,ZA,-33.9180,25.5701,58.0,
...,...,...,...,...,...,...
504,los alamos,US,35.8881,-106.3070,31.0,
517,al burayqah,LY,30.4062,19.5739,53.0,
525,gaiman,AR,-43.2897,-65.4929,27.0,
555,kenai,US,60.5544,-151.2583,74.0,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
# Import required libraries if not already imported
import hvplot.pandas
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

# Create the map without using City as a color dimension
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    color='blue',  # Use a single color instead of City
    size=50,
    alpha=0.6,
    hover_cols=['Hotel Name', 'Country', 'City']
).opts(
    tools=['hover', 'box_zoom', 'reset'],
    title='Hotel Locations'
)

# Display the map

hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country,City)